<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_Evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Imports

In [28]:
import warnings
from tqdm import tqdm
from datetime import datetime
import tensorflow as tf
tf.debugging.set_log_device_placement(False)
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from numpy import array
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from tensorflow.keras.models import load_model
from sklearn.metrics import f1_score,accuracy_score,precision_score
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

model_path = "/content/drive/My Drive/AIML-MRC-Capstone/models/"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Google Bucket as drive

In [3]:
from google.colab import auth
auth.authenticate_user()

project_id = 'ai-ml-capstone'
!gcloud config set project {project_id}
!gsutil ls


Updated property [core/project].
gs://aiml-capstone/


In [4]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  18138      0 --:--:-- --:--:-- --:--:-- 18138
OK
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


In [5]:
!mkdir gbucket
!gcsfuse --implicit-dirs aiml-capstone gbucket 
# !umount gbucket

Using mount point: /content/gbucket
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [6]:
!ls gbucket/lstmbaseline-0/tf-serve/

saved_model.pb	variables


# List of Models

As part of our capstone, we are in process of evaluating the following models

Data | Model | On GPU | Masking | Padding | Epoch | Location | 
--- | --- | --- | --- | --- | --- | --- | 
Without stopwords | SVM | No | - | - | - | [here](https://storage.cloud.google.com/aiml-capstone/svm/)
Without stopwords | LSTM Baseline | No | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/lstmbaseline-0/full_context_withoutstopwords_model_epoch_lstmbaseline0_nomask_gpu.h5)
Without stopwords | Deep LSTM + GloVe | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/deeplstm/full_context_withoutstopwords_model_epoch_deeplstm_gpu.h5)
Without stopwords | Bi-LSTM + GloVe | No | No | Pre | 10 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm/full_context_withoutstopwords_model_epoch_10_bilstm_cpu.h5)
Without stopwords | Bi-LSTM + GloVe + Q2C Attention | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm-q2c-attention-glove/full_context_withoutstopwords_nomask_epoch_25_bilstm_q2c-attention_glove_gpu.h5)
Without stopwords | Bi-LSTM + GloVe + Q2C-C2Q Attention | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm-bidaf-glove/full_context_withoutstopwords_model_epoch_25_bilstm_bidaf_glove_nomask_gpu.h5)
--- | --- | --- | --- | --- | --- | --- | 
With Stopwords | Bi-LSTM + Universal Sentence Encoder + Q2C Attention
With Stopwords | Bi-LSTM + Universal Sentence Encoder + Q2C-C2Q Attention
With Stopwords | BERT + Universal Sentence Encoder 
With Stopwords | GPT2

In [22]:
# TODO 
#     "bilstm-use-q2c-attention":{
#         "name":"Bi-LSTM + Universal Sentence Encoder + Q2C Attention",
#         "loc":""
#     },
#     "bilstm-use-q2c-c2q-attention":{
#         "name":"Bi-LSTM + Universal Sentence Encoder + Q2C-C2Q Attention",
#         "loc":""
#     },
#     "bert-use":{
#         "name":"BERT + Universal Sentence Encoder",
#         "loc":""
#     },
#     "gpt":{
#         "name":"GPT",
#         "loc":""
#     }    
# }

list_of_models = [
                  {
                    "id":"lstm-baseline",
                    "name":"LSTM Baseline",
                    "loc":"/content/drive/My Drive/AIML-MRC-Capstone/models/lstmbaseline-0/tf-serve/"
                  },
                  {
                    "id":"deeplstm-glove",
                    "name":"Deep LSTM + GloVe",
                    "loc":"/content/drive/My Drive/AIML-MRC-Capstone/models/deeplstm/tf-serve/"
                  },  
                  {
                    "id":"bilstm-glove",
                    "name":"Bi-LSTM + GloVe",
                    "loc":"/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm/full_context_withoutstopwords_model_epoch_25_bilstm_glove_nomask_gpu.h5"
                  },
                  {
                    "id":"bilstm-glove-q2c-attention",
                    "name":"Bi-LSTM + GloVe + Q2C Attention",
                    "loc":"/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-q2c-attention-glove/tf-serve/"
                  },  
                  {
                    "id":"bilstm-bidaf-glove",
                    "name":"Bi-LSTM + GloVe + Q2C-C2Q Attention",
                    "loc":"/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-bidaf-glove/tf-serve/"
                  }                                    
                  ]

list_of_models


[{'id': 'lstm-baseline',
  'loc': '/content/drive/My Drive/AIML-MRC-Capstone/models/lstmbaseline-0/tf-serve/',
  'name': 'LSTM Baseline'},
 {'id': 'deeplstm-glove',
  'loc': '/content/drive/My Drive/AIML-MRC-Capstone/models/deeplstm/tf-serve/',
  'name': 'Deep LSTM + GloVe'},
 {'id': 'bilstm-glove',
  'loc': '/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm/full_context_withoutstopwords_model_epoch_25_bilstm_glove_nomask_gpu.h5',
  'name': 'Bi-LSTM + GloVe'},
 {'id': 'bilstm-glove-q2c-attention',
  'loc': '/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-q2c-attention-glove/tf-serve/',
  'name': 'Bi-LSTM + GloVe + Q2C Attention'},
 {'id': 'bilstm-bidaf-glove',
  'loc': '/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-bidaf-glove/tf-serve/',
  'name': 'Bi-LSTM + GloVe + Q2C-C2Q Attention'}]

# Evaluation Metrics

## bAbi 

Reference Paper = https://arxiv.org/pdf/1502.05698.pdf 

GitHub - https://github.com/facebookarchive/bAbI-tasks

![alt text](https://storage.cloud.google.com/aiml-capstone/Screenshot%202020-06-20%20at%2011.11.50%20AM.png)
![alt text](https://storage.cloud.google.com/aiml-capstone/Screenshot%202020-06-20%20at%2011.11.03%20AM.png)

## SQuAD test dataset and published dev set

Refer to this eval metrics - https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/

Eval dataset from SQuAD - https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

Test Dataset from training = 

## News Domain Specific Evaluations

Test on Sample News Context, Question and Answer pairs ??

## 

# Common Functions

## Custom function for preprocessing of context and question

In [ ]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
stop_words = set(stopwords.words('english')) 

## reference 
def decontracted(phrase):
    """
    This function remooves punctuation from given sentence.
    """

    if(phrase is np.nan):
      return 'impossible'      

    try:      
      # specific
      phrase = re.sub(r"won\'t", "will not", phrase)
      phrase = re.sub(r"can\'t", "can not", phrase)

      # general
      phrase = re.sub(r"n\'t", " not", phrase)
      phrase = re.sub(r"\'re", " are", phrase)
      phrase = re.sub(r"\'s", " is", phrase)
      phrase = re.sub(r"\'d", " would", phrase)
      phrase = re.sub(r"\'ll", " will", phrase)
      phrase = re.sub(r"\'t", " not", phrase)
      phrase = re.sub(r"\'ve", " have", phrase)
      phrase = re.sub(r"\'m", " am", phrase)
      
      # string operation
      phrase = phrase.replace('\\r', ' ')
      phrase = phrase.replace('\\"', ' ')
      phrase = phrase.replace('\\n', ' ')

      phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase.lower())
    except:
      print(phrase)  
    
    return phrase

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=False):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # doc = decontracted(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

## Answer Span from Context and Answer, and reverse for predicted spans

In [35]:
def tokenize(sentence):
    """
    Returns tokenised words.
    """
    return nltk.word_tokenize(sentence)

def answer_span(context,ans):
    """
    This funtion returns anwer span start index and end index.
    """
    ans_token = tokenize(ans)
    con_token = tokenize(context)
    ans_len = len(ans_token)
    
    if ans_len!=0 and ans_token[0] in con_token:
    
        indices = [i for i, x in enumerate(con_token) if x == ans_token[0]]        
        try:

            if(len(indices)>1):
                start = [i for i in indices if (con_token[i:i+ans_len] == ans_token) ]
                end = start[0] + ans_len - 1
                return start[0],end

            else:
                start = con_token.index(ans_token[0])
                end = start + ans_len - 1
                return start,end
        except:
            return -1,-1
    else:
        return -1,-1

def span_to_answer(span, context):
  con_token = tokenize(context)  
  return ' '.join(con_token[span[0]:span[1]+1])

## Update and persist params

In [7]:
### SAVE PARAMS
# Writing to sample.json 

def updateparams():
  with open(model_path + "params.json", "w") as p: 
    p.write(json.dumps(params))
  print("params.jsop updated and can be found in ", model_path + "params.json")  

# updateparams()

In [8]:
def showparams(params):
  pprint.pprint(params)

In [9]:
def loadparams():
  with open(model_path + "params.json") as f:
    params = json.load(f)
  return params  

showparams(loadparams())

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': [26062, 16],
 'test_span_outofrange': 0,
 'train_shape': [78183, 16],
 'train_span_outofrange': 0,
 'training.batch_size': 64,
 'training.epochs': 25,
 'training.train_length': 78183,
 'training.train_steps': 1221,
 'training.val_length': 26061,
 'training.val_steps': 814,
 'val_shape': [26061, 16],
 'val_span_outofrange': 0,
 'vocab_size': 100850}


## Create a common function to generate sequences (useful in prediction)

In [25]:
# function to generate sequences withg appropiate padding
def generate_question_context_sequence(context,question,question_max_length,padding,context_max_length):
  question_seq = tokenizer.texts_to_sequences(question)
  context_seq = tokenizer.texts_to_sequences(context)
  question_seq = preprocessing.sequence.pad_sequences(question_seq,
                                                      maxlen=question_max_length,
                                                      padding=padding)
  context_seq = preprocessing.sequence.pad_sequences(context_seq,
                                                     maxlen=context_max_length,
                                                     padding=padding)
  return context_seq, question_seq

## Create a common function to predict and test

In [10]:
def predit_test(context, question):
  # get sequence for context and question
  c_ = preprocess_text(context)
  q_ = preprocess_text(question,stopword_removal=False)
  c,q = generate_question_context_sequence(c_, q_)  
  y_ = model.predict([q,c])    
  # # for i in range(26062):
  s = np.argmax(y_[0,:params['context_max_length']])
  e = np.argmax(y_[0,params['context_max_length']:])
  answer = span_to_answer((s,e),c_[0])
  
  # print(c.shape,q.shape,y_.shape,s,e,answer)  
  # print(s, e)
  return c_,q_,[s,e],y_,answer

## Load Tokenizer

In [11]:
def load_tokenizer():
  with open(model_path + "tokenizer.pkl","rb") as infile:
      tokenizer = pickle.load(infile)
  print('Vocab Loaded - ',len(tokenizer.word_index))
  return tokenizer

## Load Test Data with Ground Truth Known

In [12]:
from sklearn.utils import resample,shuffle

def load_data_withoutstopwords():
  #### NOTE THE 2 data frames's
  df_nostopwords = 'test_squad_data_final_context_withoutstopwords.csv'
  # df_withstopwords = 'squad_data_final_withstopword_withpunctuation.csv'
  test_squad_df = pd.read_csv(project_path+df_nostopwords)
  test_squad_df.drop('Unnamed: 0',axis=1,inplace=True)


  test_squad_df["answer_word_span"] = test_squad_df["answer_word_span"].apply(lambda x :eval(x))
  print(test_squad_df.info())
  return test_squad_df

## Loss function

In [13]:
def logits_loss(y_true,logits):
    """
    Custom loss function which minimises log_loss.
    Referance https://stackoverflow.com/questions/50063613/add-loss-function-in-keras
    """
    
    #y_true = tf.cast(y_true,dtype=tf.int32)
    #logits = tf.cast(logits,dtype=tf.float32)
    
    # breaking the tensor into two half's to get start and end label.
    start_label = y_true[:,:params['context_max_length']]
    end_label = y_true[:,params['context_max_length']:]
    
    # braking the logits tensor into start and end part for loss calcultion.
    start_logit = logits[:,:params['context_max_length']]
    end_logit = logits[:,params['context_max_length']:]
    
    start_loss = tf.keras.backend.categorical_crossentropy(start_label,start_logit)
    end_loss = tf.keras.backend.categorical_crossentropy(end_label,end_logit)
    
#     start_loss = tf.losses.sparse_softmax_cross_entropy(labels=start_label, logits=start_logit)
#     end_loss = tf.losses.sparse_softmax_cross_entropy(labels=end_label, logits=end_logit)
    
    # as per paer
    
    loss = start_loss + end_loss
    
    return loss

## Load Model

In [ ]:
tf.__version__

'2.2.0'

In [14]:
from tensorflow import keras
def load_mrc_model(model_path):
  custom_objects = {"logits_loss": logits_loss}
  new_model = keras.models.load_model(model_path,custom_objects=custom_objects)
  ### Check its architecture
  # new_model.summary()  
  return new_model

In [70]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = load_mrc_model('/content/drive/My Drive/AIML-MRC-Capstone/models/bilstm/full_context_withoutstopwords_model_epoch_25_bilstm_glove_nomask_gpu.h5')
  
model.summary()
# list_of_models['bilstm-glove-q2c-c2q-attention']['loc'].replace('https://storage.cloud.google.com/aiml-capstone/','gbucket/')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
QUESTION_INPUT (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
QUESTION_EMBEDDING (Embedding)  (None, 40, 300)      30255300    QUESTION_INPUT[0][0]             
__________________________________________________________________________________________________
CONTEXT_INPUT (InputLayer)      [(None, 426)]        0                                            
__________________________________________________________________________________________________
QUESTION_LSTM (Bidirectional)   (None, 256)          1140736     QUESTION_EMBEDDING[0][0]

## Load Test Data

In [15]:
def load_test_data():
  test = pd.read_csv(model_path +'test-withoutstopwords.csv')
  test.drop('Unnamed: 0',axis=1,inplace=True)
  test["answer_word_span"] = test["answer_word_span"].apply(lambda x :eval(x))
  print(test.shape)
  return test

## Create Answer Sequence for test

In [16]:
def create_answer_sequence(test, context_length):
  # for test data
  y_test = []
  for i in range(len(test)):    
      s = np.zeros(context_length,dtype = "int")
      e = np.zeros(context_length,dtype = "int")        
      start,end = test["answer_word_span"].iloc[i]    
      s[start] = 1
      e[end] = 1
      y_test.append(np.concatenate((s,e)))
  return y_test


## Predict Test

In [17]:
def predit_test(context, question):
  # get sequence for context and question
  c_ = preprocess_text(context)
  q_ = preprocess_text(question,stopword_removal=False)
  c,q = generate_question_context_sequence(c_, q_)  
  y_ = model.predict([q,c])    
  # # for i in range(26062):
  s = np.argmax(y_[0,:params['context_max_length']])
  e = np.argmax(y_[0,params['context_max_length']:])
  answer = span_to_answer((s,e),c_[0])
  
  # print(c.shape,q.shape,y_.shape,s,e,answer)  
  # print(s, e)
  return c_,q_,[s,e],y_,answer

## Create a combined y_test array having both start and end vectors in OHE

In [31]:
def combine_y(y, test_sample_size):
  # argmax is used to get the index where the max value in a list appears, and hence 
  # for every index i, we can get the place of start and end token of the max probab
  start = []
  end = []
  for i in range(test_sample_size):
      start.append(np.argmax(y[i,:params['context_max_length']]))
      end.append(np.argmax(y[i,params['context_max_length']:]))
      
  y_new = np.zeros((test_sample_size,params['context_max_length']))
  for i in range(test_sample_size):
      y_new[i,start[i]:end[i]+1] = 1
  return y_new,start,end

## Accuracy Metrics

In [19]:
def accuracy_metrics(y_true,y_pred):
  acc_score = accuracy_score(y_true,y_pred)
  macro_f1_score = f1_score(y_true,y_pred,average="macro")
  micro_f1_score = f1_score(y_true,y_pred,average="micro")
  return acc_score,macro_f1_score,micro_f1_score
  

In [20]:
accuracy_metrics([1,1,1],[0,1,1])

(0.6666666666666666, 0.4, 0.6666666666666666)

## Exact Match Count per Model

In [32]:
def checkEMMatch(test_data, start, end):
  field_names = ["True Answer",
                "True AS and AE",
                "Predict Answer",
                "Predict AS and AE"]
  result_df = pd.DataFrame(columns=field_names)
  print('Checking for equality match percentage')
  for i in tqdm(range(test.shape[0])):  
    values = [test['clean_answer'].iloc[i], 
              test['answer_word_span'].iloc[i],
              span_to_answer([start_pred[i],end_pred[i]],test['clean_context'].iloc[i]),
              (start_pred[i],end_pred[i])]
    zipped = zip(field_names, values)
    a_dictionary = dict(zipped)
    result_df = result_df.append(a_dictionary,ignore_index=True)

  ematch = result_df[result_df['Predict Answer'] == result_df['True Answer']].shape[0]  / test.shape[0]
  del result_df

  return ematch


# Evaluations

## Eval on test data

Metrics would be F1 score micro, EM, Accuracy Score

In [25]:
test = load_test_data()
test.shape[0]

(26062, 16)


26062

In [36]:
strategy = tf.distribute.MirroredStrategy()
model_results = pd.DataFrame(columns=['Model Name','Acc Score','Micro F1', 'Macro F1','EM'])
with strategy.scope():
  test = load_test_data()
  params = loadparams()
  showparams(params)
  tokenizer = load_tokenizer()
  y_test = create_answer_sequence(test,params['context_max_length'])
  # y_test was a list changing to numpy array
  y_test_fixed = np.array(y_test)
  # compute for y_test though in this case it the max of 0 and 1 for 
  # the frist half od array size for start, and rest for end
  y_test_new,_,_ = combine_y(y_test_fixed,test.shape[0])

  test_context_sequence, test_question_sequence = generate_question_context_sequence(context=test["clean_context"].values,
                                    question=test["clean_question"].values,
                                    question_max_length=params['question_max_length'],
                                    padding=params['question_pad_seq'],
                                    context_max_length=params['context_max_length']
                                    )


  for model in list_of_models:
    print('Loading model ',model['name'],' from ', model['loc'])  
    tfmodel = load_mrc_model(model['loc'])
    print(model['name'],' model successfuly. Starting eval')
    # tfmodel.summary()  
    y_prediction = tfmodel.predict([test_question_sequence,test_context_sequence])
    y_prediction_new,start_pred,end_pred = combine_y(y_prediction,test.shape[0])
    acc_score,macro_f1_score,micro_f1_score = accuracy_metrics(y_test_new,y_prediction_new)
    emmatch = checkEMMatch(test,start_pred,end_pred)
    values = [model['name'],
              acc_score,
              micro_f1_score,
              macro_f1_score,
              emmatch]
    zipped = zip(model_results.columns, values)
    a_dictionary = dict(zipped)
    model_results = model_results.append(a_dictionary,ignore_index=True)              

model_results.head(10)   
model_results.to_csv(model_path + "model_results_" + datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p") + ".csv")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
(26062, 16)
{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': [26062, 16],
 'test_span_outofrange': 0,
 'train_shape': [78183, 16],
 'train_span_outofrange': 0,
 'training.batch_size': 64,
 'training.epochs': 25,
 'training.train_length': 78183,
 'training.train_steps': 1221,
 'training.val_length': 26061,
 'training.val_steps': 814,
 'val_shape': [26061, 16],
 'val_span_outofrange': 0,
 'vocab_size': 100850}
Vocab Loaded -  100850
Loading model  LSTM Baseline  from  /content/drive/My Drive/AIML-MRC-Capstone/models/lstmbaseline-0/tf-serve/
LSTM Baseline  model successfuly. Starting eval


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
100%|██████████| 26062/26062 [02:41<00:00, 161.49it/s]


Loading model  Deep LSTM + GloVe  from  /content/drive/My Drive/AIML-MRC-Capstone/models/deeplstm/tf-serve/
Deep LSTM + GloVe  model successfuly. Starting eval


100%|██████████| 26062/26062 [02:39<00:00, 162.98it/s]


Loading model  Bi-LSTM + GloVe  from  /content/drive/My Drive/AIML-MRC-Capstone/models/bilstm/full_context_withoutstopwords_model_epoch_25_bilstm_glove_nomask_gpu.h5
Bi-LSTM + GloVe  model successfuly. Starting eval


...

Loading model  Bi-LSTM + GloVe + Q2C Attention  from  /content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-q2c-attention-glove/tf-serve/
Bi-LSTM + GloVe + Q2C Attention  model successfuly. Starting eval


100%|██████████| 26062/26062 [02:40<00:00, 162.13it/s]


Loading model  Bi-LSTM + GloVe + Q2C-C2Q Attention  from  /content/drive/My Drive/AIML-MRC-Capstone/models/bilstm-bidaf-glove/tf-serve/
Bi-LSTM + GloVe + Q2C-C2Q Attention  model successfuly. Starting eval


100%|██████████| 26062/26062 [02:38<00:00, 164.36it/s]


,Model Name,Acc Score,Micro F1,Macro F1,EM
0,LSTM Baseline,0.356112,0.228822,0.003986,0.002686
1,Deep LSTM + GloVe,0.315594,0.221054,0.003977,0.002878
2,Bi-LSTM + GloVe,0.361100,0.255439,0.003302,0.004259
3,Bi-LSTM + GloVe + Q2C Attention,0.470647,0.337377,0.001502,0.000000
4,Bi-LSTM + GloVe + Q2C-C2Q Attention,0.345944,0.260478,0.012515,0.011933


In [37]:
model_results.to_csv(model_path + "model_results_" + datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p") + ".csv")

## Out of domain handing 


### Different language 


### Totally irrelevant question given a known context from train domain

### Totally different context say from Medicine and ask answer from it

## Eval on manual input

### TEST1

In [ ]:
c='In the Mahayana, the Buddha tends not to be viewed as merely human, but as the earthly projection of a beginningless and endless, omnipresent being (see Dharmakaya) beyond the range and reach of thought. Moreover, in certain Mahayana sutras, the Buddha, Dharma and Sangha are viewed essentially as One: all three are seen as the eternal Buddha himself.'
q='in what sutras are the buddha dharma and sangha viewed as one'

# c_,q_,span,y_,answer = predit_test(test['context'].iloc[39],test['question'].iloc[39])
c_,q_,span,y_,answer = predit_test([c],[q])
print('ori c = ')
pprint.pprint(test['context'].iloc[39])
print('ori c c = ')
pprint.pprint(test['clean_context'].iloc[39])
print('ori q = ',test['clean_question'].iloc[39])
print('new c')
pprint.pprint(c_[0])
print('new q',q_)

print('predicted answer' ,answer)